Closest Node to Path in Tree

You are given a positive integer n representing the number of nodes in a tree, numbered from 0 to n - 1. 
You are also given a 2D integer array edges of length n - 1, where edges[i] = [u, v] indicates that there is a bidirectional edge between nodes u and v.

You are given another 2D integer array query of length m, where query[i] = [start, end, node]. 

For each query, you must find the node that lies on the path between start and end which is closest to node.   Distance is measured by number of edges.

If multiple nodes on the path are tied with the same minimum distance to node, return the node with the smallest index.

Return an array answer of length m, where answer[i] is the result for query i.

Constraints:  
1 <= n <= 1000  
edges.length == n - 1  
edges[i].length == 2  
0 <= u, v <= n - 1  
u != v  
1 <= query.length <= 1000  
query[i].length == 3  
0 <= start, end, node <= n - 1  
The input graph is guaranteed to be a tree.

Example:  
Input:  
n = 7  
edges = [[0,1],[0,2],[1,3],[1,4],[2,5],[2,6]]  
query = [[3,4,6], [5,6,0], [3,5,2]]  

Output:  
[1, 2, 0]

Explanation:  
For query 1: The path between 3 and 4 is [3,1,4]. The distances to node 6 are:  
3->6 = 4 edges  
1->6 = 3 edges  
4->6 = 4 edges  
Closest node is 1.

For query 2: Path between 5 and 6 is [5,2,6]. Distances to node 0 are:  
5->0 = 2  
2->0 = 1  
6->0 = 2  
Closest node is 2.

For query 3: Path between 3 and 5 is [3,1,0,2,5]. Distances to node 2 are:  
3->2 = 3  
1->2 = 2  
0->2 = 1  
2->2 = 0  
5->2 = 1  
Closest node is 2, but 0 ties with distance 1 and has smaller index. So answer is 0.

In [ ]:
from collections import defaultdict

def lca(a, b, parent_arr, depth_arr):
    # Bring a and b to same depth
    while depth_arr[a] > depth_arr[b]:
        a = parent_arr[a]
    while depth_arr[b] > depth_arr[a]:
        b = parent_arr[b]
    # Walk up until they meet
    while a != b:
        a = parent_arr[a]
        b = parent_arr[b]
    return a

def dfs(node, parent, depth, graph, depth_arr, parent_arr):
    depth_arr[node] = depth
    parent_arr[node] = parent
    for nei in graph[node]:
        if nei != parent:
            dfs(nei, node, depth+1, graph, depth_arr, parent_arr)

def distance(a, b, parent_arr, depth_arr):
    ancestor = lca(a, b, parent_arr, depth_arr)
    return depth_arr[a] + depth_arr[b] - 2 * depth_arr[ancestor]

def get_path(start, end, parent_arr):
    path_start = []
    node = start
    while node != -1:
        path_start.append(node)
        node = parent_arr[node]
    path_end = []
    node = end
    while node != -1:
        path_end.append(node)
        node = parent_arr[node]
    
    # Find common ancestor to merge path
    i = len(path_start) - 1
    j = len(path_end) - 1
    while i >= 0 and j >= 0 and path_start[i] == path_end[j]:
        i -= 1
        j -= 1
    
    # Merge paths properly
    full_path = path_start[:i+1] + path_end[j+1::-1]
    return full_path

def closestNode(n: int, edges: list[list[int]], query:list[list[int]]) -> list[int]:
    
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)

    depth_arr = [0] * n
    parent_arr = [-1] * n
    dfs(0, -1, 0, graph, depth_arr, parent_arr)
    
    result = []
    for start, end, node in query:
        path_nodes = get_path(start, end, parent_arr)
        # Find closest node on path to query node
        min_dist = float('inf')
        closest = None
        for v in path_nodes:
            dist = distance(v, node, parent_arr, depth_arr)
            if dist < min_dist or (dist == min_dist and v < closest):
                min_dist = dist
                closest = v
        result.append(closest)
    
    return result



Approach: Depth-First Preprocessing with Graph + Lowest Common Ancestor(LCA)

Insight: First build parent and depth arrays so distances between any two nodes can be computed via LCA, then reconstruct the unique path between start and end and scan for the node on that path with minimum distance to the query node. This avoids recomputing paths from scratch for every query.

Tricky Part: The main catch is correctly merging two ancestor chains to form the complete path between any two nodes. We need to find where they meet (the LCA) and then stitch them together properly. Also, breaking ties requires picking the node with the smallest index, we need to check when distances are equal.
